In [1]:
import os
from pathlib import Path
from Utils.llm.api import ask_model
from Utils.llm.config import Model, ModelProvider
import json
import time

RESULTS_BASE_PATH = os.getenv('RESULTS_REPO_PATH')

In [6]:
COMMUNICATION_PROTOCOL = """
## Communication Protocol:
{"command": "read-file", "file_path": "path to file"}
or
{"command": "file-structure", "file_paths": ["path1", "path2","..."]}
or
{"command": "write-file", "file_path": "path to file", "content": "enscaped converted code"}
or
{"command": "end"}
"""

SYSTEM_PROMPT = f"""
As an AI proficient in software engineering, particularly in Frontend development, React and Angular, 
your objective is to resolve the provided development tasks.

We will exchange command messages in JSON format according to following communication protocol.
{COMMUNICATION_PROTOCOL}

Ensure that you return only valid JSON object with only one command to execute.
Specifically, when you use the ‘write-file’ command, verify that the converted code properly transforms into a JSON string. Do not send more than one command in the message.
"""

# if you use o1 model, add {system} prompt to TASK before {objective}
TASK = """
{objective}
Outdated application is composed of the following files:
<legacy_files_structure>
{legacy_files_structure}
</legacy_files_structure>

Your task is to request content of each file and analyze its implementation.
You should ask for files content by the json command.
Example:
{{
    "command": "read-file",
    "file_path": "js/controllers/todo.js" or "app.tsx"
}}

After getting and analyzing of all file's content - think thoroughly and *return only new file structure* of translated React application in a form of JSON command.
Do not provide styles file, previous styles will be applied.
Do not provide any configs.
Example:
{{
    "command": "file-structure",
    "file_paths": [
        "name1.ts",
        "name2.tsx",
        "...",
    ]
}}
 
After you return the file structure of the updated application, 
I will request you for each file by the file_path in the list and you should use following command to give me converted file.
{instructions}

Example:
{{
    "command": "write-file",
    "file_path": "path to file"
    "content": "enscated converted code is here"
}}

Do not add any comments in generated code and follow the instructions precisely. Once you finish the whole task, please issue the "end" command.
"""


def list_files(base_path):
    base_path = Path(base_path)
    file_list = []
    for file_path in base_path.rglob('*'):
        if file_path.is_file():
            relative_path = file_path.relative_to(base_path)
            file_list.append(str(relative_path))

    return file_list


def run_experiment(task, model, read_files_amount, dataset_path, output_path, start_time):
    messages = []
    input_tokens = output_tokens = reasoning_tokens = 0
    requests_list = [{"role": "user", "content": task}]
    files_read_by_model = 0

    while True:
        if (len(requests_list) > 0):
            messages.append(requests_list.pop())
            print(f"Requests left: {len(requests_list)}")
        else:
            break

        if len(messages) > 0:
            print("REQUEST:")
            print(json.dumps(messages[-1], indent=4))

        answer = ask_model(messages, SYSTEM_PROMPT, model)
        print("RESPONSE:")
        print(json.dumps(answer, indent=4))
        command = answer["content"]
        tokens = answer["tokens"]  # {'input_tokens': 168, 'output_tokens': 2031, 'reasoning_tokens': 576}
        # Increment input_tokens, output_tokens, reasoning_tokens accordingly
        input_tokens += tokens['input_tokens']
        output_tokens += tokens['output_tokens']
        reasoning_tokens += tokens.get('reasoning_tokens', 0)
        try:
            # Gemini always return command wrapped into json markdown block
            # I don't find a way to prevent this behaviour, so cleaning json string before parsing it.
            clean_content = command
            clean_content = clean_content.strip('\n')
            clean_content = clean_content.strip('`')
            clean_content = clean_content.replace('json\n', '', 1)
            command = json.loads(clean_content)
        except json.JSONDecodeError as error:
            print(error)
            if model.provider == ModelProvider.AISTUDIO:
                messages.append({"role": "model", "content": command}) # for google models
            else:
                messages.append({"role": "assistant", "content": command})
            requests_list.append({"role": "user", "content":
                f"""Failed to parse JSON command with error: {error}.
                Ensure that you respond with valid JSON command.
                Provide only one JSON content with only ONE command to execute.
                Return only valid JSON without any comments.
                You should NEVER use markdown to wrap JSON commands."""})
            continue
        if model.provider == ModelProvider.AISTUDIO:
            messages.append({"role": "model", "content": json.dumps(command)})
        else:
            messages.append({"role": "assistant", "content": json.dumps(command)})
        if command["command"] == "end":
            print("Ending command session.")
            break
        elif command["command"] != "read-file" and files_read_by_model < read_files_amount:
            print(f"Error: Model should read all files first. Files read by model: {files_read_by_model}")
            requests_list.append({"role": "user", "content": f"Please read rest {read_files_amount - files_read_by_model} files first."})
        elif command["command"] == "read-file":
            file_path = command["file_path"]
            full_path = Path(dataset_path, file_path)
            try:
                with open(full_path, 'r') as file:
                    content = file.read()
                    requests_list.append({"role": "user", "content": content})
            except FileNotFoundError:
                print(f"Error: File at {full_path} not found.")
            files_read_by_model += 1
        elif command["command"] == "write-file":
            file_path = command["file_path"]
            full_path = Path(output_path, file_path.lstrip('/'))
            full_path.parent.mkdir(parents=True, exist_ok=True)
            content = command["content"]
            try:
                with open(full_path, 'w') as file:
                    if ".json" in file_path:
                        json.dump(content, file, indent=4)
                    else:
                        file.write(content)
                print(f"File written successfully at {file_path}.")
            except IOError as e:
                print(f"Failed to write file at {file_path}. Error: {e}")
        elif command["command"] == "file-structure":
            file_paths = command["file_paths"]
            for path in file_paths:
                requests_list.append({"role": "user", "content": f"Give me converted code of {path}"})
                print(path)
        else:
            reply = f"""Unknown command {command['command']}
            Please use only supported commands:
            {COMMUNICATION_PROTOCOL}
            """
            messages.append({"role": "user", "content": reply})
    end_time = int(time.time())
    output = {
        "messages": messages,
        "time": end_time - start_time,
        "total_tokens": {
            "input_tokens": input_tokens,
            "output_tokens": output_tokens,
            "reasoning_tokens": reasoning_tokens,
        }
    }
    messages_log = json.dumps(output, indent=4)
    messages_log_path = Path(output_path, "message_log.json")
    with open(messages_log_path, 'w') as file:
        file.write(messages_log)


In [9]:
model = Model.Grok3mini_beta
current_unix_time = int(time.time())
DATASET_PATH = f"./Dataset/JS/ToDoApp_AngularJS"
OUTPUT_PATH = f"{RESULTS_BASE_PATH}/Output/{model}/JS/contextual_experiment/translate_to_react/{current_unix_time}/"

legacy_files_list = list_files(DATASET_PATH)
legacy_files_structure = "\n".join(legacy_files_list)

OBJECTIVE = "Your task is to translate outdated AngularJS app code to recent version of React using functional components and hooks."

INSTRUCTIONS = f"""
Apply these instructions for translated application:
    - Use the following libraries: TypeScript, Redux Toolkit with createSlice, and nanoid.
    - Provide configuration of the store and provider if needed.
    - Split the code into separate components.
    - Optimize the code where possible.
    - The converted code should not contain any TODOs.
    - Return the translated code as markdown code snippets.
    - Simply return the codebase without additional comments or explanations on how to convert it.
"""

task = TASK.format(
    # system=SYSTEM_PROMPT,
    objective=OBJECTIVE,
    legacy_files_structure=legacy_files_structure,
    instructions=INSTRUCTIONS)

print(task)

run_experiment(
    task=task,
    model=model,
    read_files_amount=len(legacy_files_list),
    dataset_path=DATASET_PATH,
    output_path=OUTPUT_PATH,
    start_time=current_unix_time
)


Your task is to translate outdated AngularJS app code to recent version of React using functional components and hooks.
Outdated application is composed of the following files:
<legacy_files_structure>
index.html
js/main.js
js/app.js
js/directives/todoFocus.js
js/directives/todoEscape.js
js/controllers/todo.js
js/services/todoStorage.js
</legacy_files_structure>

Your task is to request content of each file and analyze its implementation.
You should ask for files content by the json command.
Example:
{
    "command": "read-file",
    "file_path": "js/controllers/todo.js" or "app.tsx"
}

After getting and analyzing of all file's content - think thoroughly and *return only new file structure* of translated React application in a form of JSON command.
Do not provide styles file, previous styles will be applied.
Do not provide any configs.
Example:
{
    "command": "file-structure",
    "file_paths": [
        "name1.ts",
        "name2.tsx",
        "...",
    ]
}
 
After you return the 

In [8]:
model = Model.Grok3mini_beta
current_unix_time = int(time.time())
DATASET_PATH = f"./Dataset/JS/ToDoApp_ReactJS"
OUTPUT_PATH = f"{RESULTS_BASE_PATH}/Output/{model}/JS/contextual_experiment/update_react/{current_unix_time}/"
legacy_files_list = list_files(DATASET_PATH)
legacy_files_structure = "\n".join(legacy_files_list)
print(legacy_files_list, len(legacy_files_list))

OBJECTIVE = "Your task is to translate outdated React app code to modern version of React."

INSTRUCTIONS = """
While updating application - follow next instructions:
- Use the following libraries: TypeScript, Redux Toolkit with createSlice, and nanoid.
- Provide a package.json file if possible and configure the store and provider if needed.
- Split the code into separate components.
- Optimize the code where possible.
- The converted code should not contain any TODOs.
- Return the translated code as markdown code snippets.
- Simply return the codebase without additional comments or explanations on how to convert it.
"""

task = TASK.format(
    system=SYSTEM_PROMPT,
    objective=OBJECTIVE,
    legacy_files_structure=legacy_files_structure,
    instructions=INSTRUCTIONS)

print(task)

run_experiment(
    task=task,
    model=model,
    read_files_amount=len(legacy_files_list),
    dataset_path=DATASET_PATH,
    output_path=OUTPUT_PATH,
    start_time=current_unix_time
)

['app.tsx', 'todoItem.tsx', 'utils.ts', 'footer.tsx', 'constants.ts', 'todoModel.ts'] 6

Your task is to translate outdated React app code to modern version of React.
Outdated application is composed of the following files:
<legacy_files_structure>
app.tsx
todoItem.tsx
utils.ts
footer.tsx
constants.ts
todoModel.ts
</legacy_files_structure>

Your task is to request content of each file and analyze its implementation.
You should ask for files content by the json command.
Example:
{
    "command": "read-file",
    "file_path": "js/controllers/todo.js" or "app.tsx"
}

After getting and analyzing of all file's content - think thoroughly and *return only new file structure* of translated React application in a form of JSON command.
Do not provide styles file, previous styles will be applied.
Do not provide any configs.
Example:
{
    "command": "file-structure",
    "file_paths": [
        "name1.ts",
        "name2.tsx",
        "...",
    ]
}
 
After you return the file structure of the up